In [1]:
!pip install ipyleaflet
!pip install ipywidgets
!pip install ipyvolume
!pip install altair
!pip install vega_datasets
!pip install bqplot
!pip install nbdime
!pip install palettable
!pip install earthengine-api
!pip install folium

In [2]:
%%bash
jupyter labextension install jupyter-leaflet
jupyter labextension install @jupyter-widgets/jupyterlab-manager
jupyter nbextension install --py --symlink --sys-prefix ipyleaflet
jupyter nbextension enable --py --sys-prefix ipyleaflet
jupyter labextension install js
jupyter labextension install ipyvolume
jupyter nbextension enable --py --sys-prefix widgetsnbextension
jupyter labextension install @jupyter-widgets/jupyterlab-manager
jupyter nbextension enable --py widgetsnbextension --sys-prefix
jupyter labextension install bqplot
jupyter nbextension install bqplot --py --symlink --sys-prefix
jupyter nbextension enable --py --sys-prefix bqplot

Couldn't find program: 'bash'


In [3]:
import ee # Google Earth Engine
import folium # help visualize data
import geehydro #Python package for mapping inundation dynamics using Google Earth Engine
from IPython.display import Image
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import bqplot
import datetime
import dateutil.parser
import ipywidgets
import ipyleaflet
import IPython.display
import pprint
import pandas as pd
import traitlets

# Configure the pretty printing output & initialize earthengine.
pp = pprint.PrettyPrinter(depth=4)
ee.Authenticate()
ee.Initialize()




Enter verification code: 4/zwGGtsAv5rXy7YTBq-9hub09OoxyUlx6F02z5W776Yus6O9clf3TE-o

Successfully saved authorization token.


In [4]:
#ee.Authenticate()
ee.Initialize()

In [4]:
# Function to get sizes in Human readable format
suffixes = ['B', 'KB', 'MB', 'GB', 'TB', 'PB']
def humansize(nbytes):
    i = 0
    while nbytes >= 1024 and i < len(suffixes)-1:
        nbytes /= 1024.
        i += 1
    f = ('%.2f' % nbytes).rstrip('0').rstrip('.')
    return '%s %s' % (f, suffixes[i])

In [5]:
#Public Image Collections Landset 8
# this is globle, there is no geometry filter here.
# cloud cover is less than 2%
l8sr = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterDate('2018-08-01','2018-12-01').filterMetadata('CLOUD_COVER','less_than',2)

# Private Image Collection
# planet scope dataset, over the top of california.
ps4bsr=ee.ImageCollection('projects/sat-io/open-ca/ps4bsr').filterDate('2018-08-01','2018-12-01')

# Get collection size
print('Total number of LS-8 assets with filters: '+str(l8sr.size().getInfo()))
print('Total number of PS assets with filters: '+str(ps4bsr.size().getInfo()))
print('\n'+'Total size of LS-8 collection : '+str(humansize(l8sr.reduceColumns(ee.Reducer.sum(), ['system:asset_size']).getInfo()['sum'])))
print('Total size of PS collection : '+str(humansize(ps4bsr.reduceColumns(ee.Reducer.sum(), ['system:asset_size']).getInfo()['sum'])))

Total number of LS-8 assets with filters: 10681
Total number of PS assets with filters: 1046

Total size of LS-8 collection : 5.68 TB
Total size of PS collection : 161.51 GB


In [6]:
# Get sample image from collection
# want to get meta data
# Using pp.pprint to make it easier to read the metadata
sample_image = ee.Image(ps4bsr.first())
pp.pprint(sample_image.getInfo())
band_names_original = sample_image.bandNames()
band_names_original.getInfo()



{'bands': [{'crs': 'EPSG:32610',
            'crs_transform': [3, 0, 597810, 0, -3, 4145118],
            'data_type': {'max': 65535,
                          'min': 0,
                          'precision': 'int',
                          'type': 'PixelType'},
            'dimensions': [8918, 4383],
            'id': 'b1'},
           {'crs': 'EPSG:32610',
            'crs_transform': [3, 0, 597810, 0, -3, 4145118],
            'data_type': {'max': 65535,
                          'min': 0,
                          'precision': 'int',
                          'type': 'PixelType'},
            'dimensions': [8918, 4383],
            'id': 'b2'},
           {'crs': 'EPSG:32610',
            'crs_transform': [3, 0, 597810, 0, -3, 4145118],
            'data_type': {'max': 65535,
                          'min': 0,
                          'precision': 'int',
                          'type': 'PixelType'},
            'dimensions': [8918, 4383],
            'id': 'b3'},
           {'

['b1', 'b2', 'b3', 'b4']

In [7]:
sample_image_2 = ee.Image(l8sr.first())
pp.pprint(sample_image_2.getInfo())
band_names_original = sample_image_2.bandNames()
band_names_original.getInfo()

{'bands': [{'crs': 'EPSG:32628',
            'crs_transform': [30, 0, 341385, 0, -30, 8807715],
            'data_type': {'max': 32767,
                          'min': -32768,
                          'precision': 'int',
                          'type': 'PixelType'},
            'dimensions': [9161, 9161],
            'id': 'B1'},
           {'crs': 'EPSG:32628',
            'crs_transform': [30, 0, 341385, 0, -30, 8807715],
            'data_type': {'max': 32767,
                          'min': -32768,
                          'precision': 'int',
                          'type': 'PixelType'},
            'dimensions': [9161, 9161],
            'id': 'B2'},
           {'crs': 'EPSG:32628',
            'crs_transform': [30, 0, 341385, 0, -30, 8807715],
            'data_type': {'max': 32767,
                          'min': -32768,
                          'precision': 'int',
                          'type': 'PixelType'},
            'dimensions': [9161, 9161],
            'id':

['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B10',
 'B11',
 'sr_aerosol',
 'pixel_qa',
 'radsat_qa']

In [8]:
# Function to get tilelayer url from earthengine server
# this allows us to render directly from earth engine
def GetTileLayerUrl(ee_image_object):
  map_id = ee.Image(ee_image_object).getMapId()
  tile_url_template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
  return tile_url_template.format(**map_id)

In [9]:
#Create a slider widget to add both Landsat 8 and PlanetScope 4B SR imagery
map1 = ipyleaflet.Map(
    center=(37.4112,-122.0634), zoom=12,
    layout={'height':'500px'},
)

ps4bsr_tile_url=GetTileLayerUrl(ps4bsr.median().visualize(min=600, max=4000,
                bands=['b4', 'b3', 'b2']))
l8sr_tile_url = GetTileLayerUrl(l8sr.median().visualize(min=100, max=3500,
                gamma=1.5, bands= ['B5', 'B3', 'B2']))  #Landsat 8 SR
left = ipyleaflet.TileLayer(url=ps4bsr_tile_url)
right=ipyleaflet.TileLayer(url=l8sr_tile_url)
control = ipyleaflet.SplitMapControl(left_layer=left, right_layer=right)
map1.add_control(control)
map1

Map(center=[37.4112, -122.0634], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', '…

In [8]:
#Load an image.
image = ee.Image('LANDSAT/LC08/C01/T1/LC08_044034_20140318');

In [9]:
#Get information about the bands as a list.
bandNames = image.bandNames()
print('Band names: '+str(bandNames.getInfo())) # ee.List of band names

Band names: ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'BQA']


In [11]:
#Get projection information from band 1.
b1proj = image.select('B1').projection()
print('Band 1 projection: '+str(b1proj.getInfo())) # ee.Projection object

Band 1 projection: {'type': 'Projection', 'crs': 'EPSG:32610', 'transform': [30, 0, 460785, 0, -30, 4264215]}


In [12]:
#Get scale (in meters) information from band 1.
b1scale = image.select('B1').projection().nominalScale()
print('Band 1 scale: '+str(b1scale.getInfo())) # ee.Number

Band 1 scale: 30
